# Tumor Classification

Authors: Ben Shealy, Colin Targonski

In this notebook, we'll show you some examples of using machine learning techniques in the domain of __bioinformatics__. Specifically, we're going to use a neural network to classify several tumor samples by tumor type and tumor stage.

## Background

While machine learning and deep learning have recently found great success in fields such as computer vision and speech recognition, it is becoming increasingly apparent that there is a great need for machine learning expertise in health-related fields such as bioinformatics. The main reason is that biological datasets are _huge_ and _very complex_. For example, the human __genome__ consists of over 3 billion __base pairs__ (A/T/C/G's), which means that the DNA sequence for _a single person_ is a ~3 GB file! On top of that, DNA is just one type of data that we can extract from humans and other organisms -- there are many other types as well:

- Genomic (DNA)
- Transcriptomic (RNA)
- Proteomic (proteins)
- DNA methylation

These data types together are typically referred to as __omic data__, and they are probably the most commonly used types of data in machine learning applications because they are structured like a conventional dataset, with features and samples and labels, etc. Rather than work with the sequences directly, researchers typically preprocess this data into a form which is more amenable for machine learning purposes.

For example, consider DNA and RNA. The difference between these two is that while DNA contains the entire genome and is the same in all cells, RNA contains only a subset of the DNA that is used for a particular cell. Therefore, RNA is different for every type of cell in your body. DNA and RNA contain __genes__, which are specific sequences of base pairs that are used by enzymes to create proteins. The human genome contains more than 60,000 unique genes, and each gene occurs many times in the genome. We can count how many times a gene occurs in DNA or RNA, which is used to quantify the gene's __expression__ level. Therefore, while gene expression is constant in DNA across all cells, gene expression in RNA will vary based on the cell type. For example, a skin cell and a brain cell will have different RNA expression patterns, and a cancerous cell will have a different expression pattern compared to a healthy counterpart.

## Getting Started

In this notebook we're going to work with RNA expression data derived from kidney tumor samples. This data is taken 
from __The Cancer Genome Atles (TCGA)__ project, which contains RNA sequences for a wide array of cancers. Our dataset contains samples from three types of kidney cancer -- KICH, KIRC, and KIRP -- as well as normal samples which were taken from heathly parts of the same kidneys. This dataset is available on our [Box folder](https://clemson.app.box.com/folder/11145145746), or it can be extracted from the original TCGA data (consult the [AIZheng](https://github.com/SVAI/AIzheng) project).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.manifold
import sklearn.model_selection
from tensorflow import keras

## Loading the Data

The dataset consists of two files:

- `kidney.emx.txt`: expression matrix with genes as rows and samples as columns
- `kidney.labels.txt`: label file containing various labels (tumor type, tumor stage, etc.) for each sample

We can load each of these files easily as pandas dataframes:

In [ ]:
data = pd.read_csv("kidney.emx.txt", index_col=0, sep="\t")
labels = pd.read_csv("kidney.labels.txt", index_col=0, sep="\t")

print(data.shape)

In [ ]:
data.head()

In [ ]:
labels.head()

As you can see, the dataset contains 60,483 genes (rows) and 1,009 samples (columns), and for each sample there is a label for the tumor type and tumor stage.

The `tumor_state` column denotes whether the sample was taken from the tumorous part or the healthy part of the kidney. For example, the third row is a sample from the _healthy_ part of a KICH stage 3 tumor. These healthy samples should allow us to make a direct comparison between "healthy" and "tumorous" expression patterns.

We need to perform a few preprocessing steps so that our data is compatible with scikit-learn. In particular, the data matrix needs to be arranged as samples x genes instead of genes x samples, and we have to deal with missing values (NaN) in the data by either removing them or filling them with some value. In this case we will simply fill missing values with the smallest expression value in the dataset, because that value is roughly equivalent in meaning.

Also, since we have multiple labels, we need to construct a single label vector. For now, we will simply combine `tumor_type` and `tumor_state` into a single label, which will have $3 \times 2 = 6$ classes.

In [ ]:
# transpose data, fill missing values
X = data.T
X = X.fillna(X.min().min())

# construct label vector from tumor_type
y = labels["tumor_type"].copy()
y[labels["tumor_state"] == 0] = "normal"

## Visualizing the Data

Now let's create a few visualizations of the dataset to better understand it.

First, we need to understand how the samples are broken down by tumor type, tumor stage, and tumor state (normal vs. tumor). To do this we can use seaborn's `catplot` function:

In [ ]:
g = sns.catplot(x="tumor_stage", hue="tumor_state", col="tumor_type", data=labels, kind="count")

From this plot we can see a couple of things: (1) there are far fewer KICH samples compared to the other two types, and (2) there are many more normal samples than there are tumor samples. In other words, this dataset is highly __imbalanced__, which will likely make things difficult when we try to do classification.

Next, let's use t-SNE to see how well the data naturally separates into groups by the various labels:

In [ ]:
# compute t-SNE of data
X_tsne = sklearn.manifold.TSNE(n_components=2).fit_transform(X)

# plot embedding, color by tumor type
plt.axis("off")

classes = list(set(y))

for c in classes:
    mask = (y == c)
    plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1], label=c)

plt.legend()
plt.show()

Finally, let's try to understand what individual genes look like. To do this, we'll take a random set of genes and plot their distributions:

In [ ]:
# define the size of the grid
rows = 4
cols = 4

# select a random set of genes
indices = np.random.choice(X.columns, rows * cols)

# plot the images in a grid
plt.figure(figsize=(4 * cols, 4 * rows))

for i in range(rows * cols):
    gene = indices[i]

    ax = plt.subplot(rows, cols, i + 1)
    sns.distplot(X[gene])
    plt.title(gene)
    plt.xlabel("")

From these gene distributions it seems that most genes are normally distributed, but some genes have more interesting profiles, such as multiple modes or skews. Because there are over 60,000 genes in this dataset, it will be difficult for any machine learning algorithm to extract the most important information from all of these features. Trying to select the most salient genes, either by variance plots or PCA or other feature selection methods, is an entire research problem on its own.

## Classifying the Data

Now, using the concepts we developed in the supervised learning and neural network notebooks, let's train a MLP to classify the samples by tumor type.

In [ ]:
classes = sorted(list(set(y)))
y_onehot = [classes.index(y_i) for y_i in y]
y_onehot = keras.utils.to_categorical(y_onehot, num_classes=len(classes))

# create train and test sets
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y_onehot, test_size=0.3)

In [ ]:
# create a 3-layer neural network
mlp = keras.models.Sequential()
mlp.add(keras.layers.Dense(units=1024, activation="sigmoid", input_shape=(X.shape[1],))) 
mlp.add(keras.layers.Dense(units=1024, activation="sigmoid"))
mlp.add(keras.layers.Dense(units=len(classes), activation="softmax"))

# compile the model
mlp.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])

# print a summary of the model
mlp.summary()

In [ ]:
# train the model
history = mlp.fit(x=X_train, y=y_train, batch_size=32, epochs=20, validation_split=0.1)

In [ ]:
# plot the training accuracy
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.title("Training Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

# plot the training loss
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Training Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

In [ ]:
# evaluate the model on the test set
scores = mlp.evaluate(x=X_test, y=y_test) 

# print results
for name, score in zip(mlp.metrics_names, scores):
    print("%s: %g" % (name, score))

In [ ]:
# get the raw predictions of the network on the test set
y_pred = mlp.predict(X_test)

# convert test labels and predicted labels from one-hot to numerical
y_test_argmax = np.argmax(y_test, axis=1)
y_pred_argmax = np.argmax(y_pred, axis=1)

# create a confusion matrix from the class predictions
cnf_matrix = sklearn.metrics.confusion_matrix(y_test_argmax, y_pred_argmax)

sns.heatmap(cnf_matrix, annot=True, fmt="d", cbar=False, square=True, xticklabels=classes, yticklabels=classes)
plt.ylabel("Expected")
plt.xlabel("Measured")
plt.title("Confusion Matrix")
plt.show()